In [1]:
# Librería estándar
import argparse
import os
import warnings
import glob
# Librerías de terceros
import joblib
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    hamming_loss,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sagemaker import Session, image_uris
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
def read_data(file_path):
    """
    Lee un archivo parquet y lo carga en un DataFrame de pandas.

    Parámetros
    ----------
    file_path: str
        Ruta al archivo parquet que se desea leer.

    Retorna
    -------
    pandas.DataFrame
        DataFrame con los datos cargados desde el archivo parquet.
    """
    data = pd.read_parquet(file_path)
    return data

In [3]:
def col_vectorizer_tfidf(df, col):
    """
    Convierte la columna especificada a string, la vectoriza usando TF-IDF y retorna la matriz y el vectorizador.

    Parámetros
    ----------
    df: pandas.DataFrame
        DataFrame que contiene la columna a vectorizar.
    col: str
        Nombre de la columna de texto.

    Retorna
    -------
    X : scipy.sparse.csr.csr_matrix
        Matriz TF-IDF.
    vectorizer : TfidfVectorizer
        Vectorizador entrenado.
    """
    df[col] = df[col].astype(str)
    vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
    X = vectorizer.fit_transform(df[col].str.lower())
    return X, vectorizer


In [4]:
def print_score(y_pred, clf):
    """
    Imprime métricas de evaluación de clasificación multilabel para un clasificador dado.

    Parámetros
    ----------
    y_pred : array-like
        Predicciones generadas por el clasificador.
    clf : objeto clasificador
        Instancia del modelo utilizado para predecir, se usa solo para mostrar su nombre de clase.

    Comportamiento
    -------------
    - Muestra por consola:
      * Nombre de la clase del clasificador.
      * Exactitud (accuracy).
      * Recall ponderado.
      * Precisión ponderada.
      * Puntuación F1 ponderada.
      * Puntuación de Jaccard promedio (porcentaje).
      * Hamming loss en porcentaje.
    - Asume que la variable global `y_test` contiene las etiquetas verdaderas.

    Retorna
    -------
    None
    """
    print("Clf: ", clf.__class__.__name__)
    print("Accuracy score: {}".format(accuracy_score(y_test, y_pred)))
    print("Recall score: {}".format(recall_score(y_true=y_test, y_pred=y_pred, average='weighted')))
    print("Precision score: {}".format(precision_score(y_true=y_test, y_pred=y_pred, average='weighted')))
    print("F1 score: {}".format(f1_score(y_pred, y_test, average='weighted')))
    print("Jacard score: {}".format(avg_jacard(y_test, y_pred)))
    print("Hamming loss: {}".format(hamming_loss(y_pred, y_test)*100))
    print("---")


In [5]:
def predict_top_n_tags(text, vectorizer, classifier, multilabel_binarizer, n=5):
    """
    Predice las n etiquetas más probables para un texto dado.

    Parámetros:
    -----------
    text : str
        Texto para el cual predecir etiquetas.
    vectorizer : TfidfVectorizer
        Vectorizador TF-IDF entrenado.
    classifier : OneVsRestClassifier
        Clasificador entrenado.
    multilabel_binarizer : MultiLabelBinarizer
        Binarizador de etiquetas utilizado.
    n : int
        Número máximo de etiquetas a predecir.

    Retorna:
    --------
    list
        Lista de las n etiquetas más probables.
    """
    # Transformar el texto usando el vectorizador
    X = vectorizer.transform([text.lower()])

    # Obtener probabilidades para cada clase
    y_proba = classifier.predict_proba(X)

    # Obtener los índices de las n probabilidades más altas
    top_n_indices = y_proba[0].argsort()[-n:][::-1]

    # Crear matriz binaria para las etiquetas top
    y_pred = np.zeros(y_proba.shape[1], dtype=int)
    for idx in top_n_indices:
        # Solo incluir si la probabilidad supera un umbral mínimo (opcional)
        if y_proba[0][idx] > 0.05:
            y_pred[idx] = 1

    # Convertir de matriz binaria a etiquetas
    predicted_tags = multilabel_binarizer.inverse_transform(y_pred.reshape(1, -1))

    return predicted_tags[0]


In [6]:
data = pd.read_parquet("s3://prueba-tecnica-nequi-camilo/data/train/")

In [7]:
data

,Id,Title,Body,Tags
0,330,None,None,"[c++, class]"
1,330,None,None,"[c++, class]"
2,510170,None,None,[windows]
3,2530,None,None,[html]
4,330.686029339009,None,None,"[c++, class]"
...,...,...,...,...
740634,9997520,Using OPENXML in SQL Server 2008 stored proc -...,I'm using SQL Server 2008's XML-parsing abilit...,"[sql, xml, sql-server-2008]"
740635,9998110,What is the meaning of 'idx_categories_desc_ca...,while working on osCommerce-3 i got the table ...,[mysql]
740636,9998260,css absolute position won't work with margin-l...,Say you have the following css applied to a di...,"[html, css, html5, css3]"
740637,9999540,How to improve SQLAlchemy performance?,I made a client application that uses HTTP to ...,"[python, sql]"


In [8]:
data['text'] = data['Title'] + " " + data['Body']

In [9]:
X, vectorizer = col_vectorizer_tfidf(data, 'text')

In [10]:
y = data['Tags']

In [11]:
multilabel_binarizer = MultiLabelBinarizer()

In [12]:
new_y = multilabel_binarizer.fit_transform(y)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, new_y)

In [14]:
pipeline = Pipeline([
    ("clf", OneVsRestClassifier(
        LogisticRegression(max_iter=300)
    ))
])

In [15]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(max_iter=300)))])

In [16]:
data

,Id,Title,Body,Tags,text
0,330,None,None,"[c++, class]",nan
1,330,None,None,"[c++, class]",nan
2,510170,None,None,[windows],nan
3,2530,None,None,[html],nan
4,330.686029339009,None,None,"[c++, class]",nan
...,...,...,...,...,...
740634,9997520,Using OPENXML in SQL Server 2008 stored proc -...,I'm using SQL Server 2008's XML-parsing abilit...,"[sql, xml, sql-server-2008]",Using OPENXML in SQL Server 2008 stored proc -...
740635,9998110,What is the meaning of 'idx_categories_desc_ca...,while working on osCommerce-3 i got the table ...,[mysql],What is the meaning of 'idx_categories_desc_ca...
740636,9998260,css absolute position won't work with margin-l...,Say you have the following css applied to a di...,"[html, css, html5, css3]",css absolute position won't work with margin-l...
740637,9999540,How to improve SQLAlchemy performance?,I made a client application that uses HTTP to ...,"[python, sql]",How to improve SQLAlchemy performance? I made ...


In [17]:
clf = OneVsRestClassifier(LogisticRegression(solver='liblinear'))
clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression(solver='liblinear'))

In [ ]:
import tarfile
joblib.dump(clf, "model.joblib")
with tarfile.open("model.tar.gz", "w:gz") as tar:
    tar.add("model.joblib")

In [ ]:
from sagemaker import Session
sess = Session()
model_s3 = sess.upload_data("model.tar.gz", 
                            bucket="prueba-tecnica-nequi-camilo",
                            key_prefix="model-artifacts")

In [ ]:
from sagemaker.sklearn.model import SKLearnModel
from sagemaker import Session
from sagemaker import get_execution_role

sess = Session()
role = get_execution_role()
prefix = "model-artifacts"
model_artifact = f"s3://prueba-tecnica-nequi-camilo/{prefix}/model.tar.gz"

sklearn_model = SKLearnModel(
    model_data=model_artifact,
    role=role,
    framework_version="0.23-1",
    py_version="py3",
    sagemaker_session=sess
)

In [ ]:
model_package_arn = sklearn_model.register(
    model_package_group_name="PruebaTecnica",
    content_types=["tapplication/x-parquet"],
    response_types=["application/json"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"]
)
print("Registered ModelPackage ARN:", model_package_arn)

In [ ]:
predictor = sklearn_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large"
 )


In [29]:
data_muestra = read_data('s3://prueba-tecnica-nequi-camilo/inferencia/new_data/muestras_100_preguntas.parquet')

In [31]:
data_muestra['text'] = data_muestra['Title'] + ' ' + data_muestra['Body'].fillna('')
X_muestra = vectorizer.transform(data_muestra['text'].astype(str).str.lower())

In [32]:
resultados = []
for idx, row in data_muestra.iterrows():
    # Combinar título y cuerpo
    texto = row['text']

    # Predecir etiquetas
    tags_pred = predict_top_n_tags(texto, vectorizer, clf, multilabel_binarizer, n=5)

    # Guardar resultados
    resultados.append({
        'ID': row['Id'],
        'Título': row['Title'],
        'Etiquetas_reales': row['Tag'],
        'Etiquetas_predichas': tags_pred,
        'Acierto': any(tag in row['Tag'] for tag in tags_pred) if len(tags_pred) > 0 else False
    })

In [34]:
df_resultados = pd.DataFrame(resultados)
print("\n--- Resumen de predicciones ---")
print(f"Total de muestras: {len(df_resultados)}")
print(f"Precisión global: {df_resultados['Acierto'].mean():.2f}")



--- Resumen de predicciones ---
Total de muestras: 100
Precisión global: 0.96


In [35]:
print("\n--- Ejemplos de predicciones ---")
for i in range(min(5, len(df_resultados))):
    print(f"\nEjemplo {i+1}:")
    print(f"Título: {df_resultados.iloc[i]['Título']}")
    print(f"Etiquetas reales: {df_resultados.iloc[i]['Etiquetas_reales']}")
    print(f"Etiquetas predichas: {df_resultados.iloc[i]['Etiquetas_predichas']}")


--- Ejemplos de predicciones ---

Ejemplo 1:
Título: Binary heap insertion, don't understand for loop
Etiquetas reales: ['algorithm']
Etiquetas predichas: ('algorithm', 'arrays', 'c', 'c++', 'java')

Ejemplo 2:
Título: WebApi dynamic object not binding
Etiquetas reales: ['c#' 'jquery']
Etiquetas predichas: ('ajax', 'asp.net-mvc', 'javascript', 'jquery', 'json')

Ejemplo 3:
Título: How to check the row & column of portlet in portlet layout
Etiquetas reales: ['java']
Etiquetas predichas: ('android', 'css', 'html', 'java', 'javascript')

Ejemplo 4:
Título: Preferring dictionaries over objects in Python
Etiquetas reales: ['python']
Etiquetas predichas: ('c++', 'python', 'qt')

Ejemplo 5:
Título: GAE efficient Datastore
Etiquetas reales: ['java']
Etiquetas predichas: ('java', 'python')
